# Intermediate PyTorch: Building architectures

## Every neural net requires the `init` method and the `forward` method. 
`init` : Define the kinds of layers from PyTorch you want to utilize
`forward` : Define the input feature matrix interaction with the layers

In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms


### Building a model class

In [4]:

class CustomNet(nn.Module): # nn.Module is the base class for all neural network modules
    ## Copilot recommended
    def __init__(self):
        super(CustomNet, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 3)
        self.fc3 = nn.Linear(3, 1)

    def forward(self, x): # Defines the computation performed at every call , every neuron
        # Forward pass
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))
        return x

    def predict(self, x):
        # Predicting the output
        pred = self.forward(x)
        return pred

    def get_weights(self):
        # Get the weights
        return self.fc1.weight, self.fc2.weight

    def get_bias(self):
        # Get the bias
        return self.fc1.bias, self.fc2.bias

### Preventing Exploding Gradients and Vanishing Gradients 
 - Use Batch Normalization : During training, we normalize the values of each layer neuron
 - Use elu instead of relu to prevent non-zero gradients for 0 valued outputs

In [5]:
# Sample Batch norm
class CustomNet(nn.Module): # nn.Module is the base class for all neural network modules
    ## Copilot recommended
    def __init__(self):
        super(CustomNet, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.bn1 = nn.BatchNorm1d(5)
        self.fc2 = nn.Linear(5, 3)
        self.bn2 = nn.BatchNorm1d(3)
        self.fc3 = nn.Linear(3, 1)

    def forward(self, x): # Defines the computation performed at every call , every neuron
        x = F.elu(self.fc1(x))
        x = self.bn1(x)
        x = F.elu(self.fc2(x))
        x = self.bn2(x)
        x = F.sigmoid(self.fc3(x))
        return x
    
    
    

### Convolution Neural Networks for images.
Why not linear layers ? Considering a single image as rowPixel x colPixel. 256 * 256 = ..some big number. Passed through this layer can cause large computation time for gradient updates and processing power as well.

Convolution makes use of a filter which is moved over the original image to extract it's feature map. Padding of 0 on the border of image is done to maintain spatial dimension and avoid information loss.

Usual process :

Image --> Convolution --> Activation --> MaxPooling( to reduce dimension ) --> Convolution --> Activation --> MaxPooling( to reduce dimension ) --> Fully Connected Linear layer

- `Convolution` : Changes number of output channels
- `MaxPool`     : Reduces the size of each channel

In [6]:
class Convolution_nn(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_map = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1) ,# 3 input channels, 32 output channels, 3x3 kernel
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),)
        self.classifier = nn.Linear(64*16*16, num_classes) # 64*8*8 is the number of features extracted by the convolutional layers

    def forward(self, x):  
        # Pass input interactions
        x = self.feature_map(x)
        x = self.classifier(x)
        return x

### Data Augmentation: Transforms you can use on a datasets


In [7]:
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(35),
    transforms.RandomAutocontrast(),
    transforms.ToTensor(),
    transforms.Resize((64,64)),
])

### Metrics in torchmetrics

Precision measures the accuracy of positive predictions. The formula is:


$Precision = \frac{True Positives (TP)}{True Positives (TP) + False Positives (FP)}$


Recall measures the ability of a model to find all the relevant cases (true positives). The formula is:


$Recall = \frac{True Positives (TP)}{True Positives (TP) + False Negatives (FN)}$


In [10]:
from torchmetrics import Precision, Recall

metric_precision = Precision(task="multiclass", num_classes=7, average="micro")
metric_recall = Recall(task="multiclass", num_classes=7, average="micro")



### RNN model : Recurrent Neural Network
For time series, Genrative models, etc
Hidden layer of previous layer passed as input to next layer as well.
### Deep RNN , stacking multiple layers is also possible

In [14]:
#Basic RNN Model
class Rnn_(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.rnn = nn.RNN(input_size=10, hidden_size=5, num_layers=2, batch_first=True)
        self.linear = nn.Linear(5, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 5)
        out_, _ = self.rnn(x, h0)
        out_ = self.linear(out_[:, -1, :])
        return x
    

### RNN is short term memory. For long term memory LSTM and GRU


In [15]:
#Basic LSTM Model
class Lstm_(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size=10, hidden_size=5, num_layers=2, batch_first=True)
        self.linear = nn.Linear(5, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 5)
        c0 = torch.zeros(2, x.size(0), 5)
        out_, _ = self.lstm(x, (h0, c0))
        out_ = self.linear(out_[:, -1, :])
        return x


In [16]:
#Basic GRU Model
class Gru_(nn.Module):
    def __init__(self,input_size):
        super().__init__()
        self.gru = nn.GRU(input_size=10, hidden_size=5, num_layers=2, batch_first=True)
        self.linear = nn.Linear(5, 1)

    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), 5)
        out_, _ = self.gru(x, h0)
        out_ = self.linear(out_[:, -1, :])
        return x
    

In [ ]:
# For time series data, the input to the model is expected in (batch, seq_len, features) format.
# Accordingly change training loop / eval loop as follows:
for epoch in range(num_epoch):
    for batch in train_loader:
        x, y = batch
        x = x.permute(0, 2, 1) # permute the input to (batch, seq_len, features)
        # rest of the training loop
        # ...
        # ...




#### Multiple Input, Multiple output models

- Multi input, Single Ouput : Image , text as input with the classified label as output
- Single input, Multiple Output : Image used to provide information on multiple objects in the image.
- Regularization : Take outputs at each NN and pass it for further learning of representation

In [17]:
# Multi-input, Single Output 
# Note : If Single-input, Multi-output, then use 2 classifer layers in init and return 2 outputs in forward
class Net_(nn.Module):
    def __init__(self):
        super().__init__()
        self.img_layer = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),   # 3 input channels, 32 output channels, 3x3 kernel
                                        nn.ELU(),
                                        nn.MaxPool2d(kernel_size=2),               
                                        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), # 32 input channels, 64 output channels, 3x3 kernel
                                        nn.ELU(),
                                        nn.MaxPool2d(kernel_size=2),
                                        nn.Flatten(),
                                        nn.Linear(64*16*16, 128))                 # 64*16*16 is the number of features extracted by the convolutional layers
        self.text_layer = nn.Sequential(nn.Linear(10, 5),nn.ELU())                  # 10 input features, 5 output features  
        self.classifier = nn.Sequential(nn.Linear(128+5, 1000))                       # 128+5 is the number of features from both, 1000 is number of classes     

    def forward(self, img, text):
        img = self.img_layer(img)
        text = self.text_layer(text)
        x = torch.cat((img, text), dim=1)
        x = self.classifier(x)
        return x